## `__new__`和`__init__`
- 新式类才有`__new__`，Python3都是继承了object的
- `__new__`是用来控制对象的生成过程，在对象生成前执行
- `__init__`则是在`__new__`的基础上完善对象
- `__new__`在`__init__`之前调用
- 参数：`__new__`至少有一个参数`cls`，cls是当前类，此参数在实例化时，python解释器自动识别
- 返回值：`__new__`是必须有返回值的，且这个返回值是`__init__`的第一个参数`self`;`__init__`不需要返回值
- `__new__`不返回对象，则`__init__`不会执行；如果返回其它类了`__init__`也不执行

### 1. 查看调用顺序

In [1]:
class Demo1:
    
    def __new__(cls, *args, **kwargs):
        print("Demo1执行: __new__",cls, *args, **kwargs)
        result = super().__new__(cls)
        print("Demo1:__new__的返回值id：", id(result))
        return result
    
    def __init__(self):
        print("Demo1执行: __init__")
        print("Demo1:self的id是：", id(self))

In [2]:
# 实例化
d1 = Demo1()

Demo1执行: __new__ <class '__main__.Demo1'>
Demo1:__new__的返回值id： 4354970640
Demo1执行: __init__
Demo1:self的id是： 4354970640


> **发现：**  
1. `__new__`在`__init__`之前调用
2. 通过id：可发现`__init__`的`self`，是`__new__`的返回值

### 2. `__new__`的作用
> 当自定义的类，继承一些不可变的`class`时(eg: `int`, `str`, `tuple`)的时候，  
`__new__`提供给我们一个自定义这些类的实例化过程的途径。  
**示例：**这里我们实现一个总正数的int子类

In [3]:
class MyInt(int):
    
    def __new__(cls, value):
        print("__new__:", value)
        return super().__new__(cls, abs(value))

In [4]:
# 实例化
i1 = MyInt(-100)
i2 = MyInt(200)
i3 = MyInt(-300)

__new__: -100
__new__: 200
__new__: -300


In [5]:
print(i1, i2, i3)

100 200 300


> **发现我们无论传入的是负数, 它会变成正数**

### 3. 通过`__new__`实现单例

In [6]:
class Single:
    
    def __new__(cls, *args, **kwargs):
        print("执行: __new__")
        # 判断类是否有某个属性
        if not hasattr(cls, "instance"):
            # 表示是第一次实例化对象
            cls.instance = super().__new__(cls)
            
        # 记得返回, 只有返回了实例，才会继续调用__init__
        return cls.instance
        
    def __init__(self, name):
        print("执行__init__, name =", name)
        self.name = name

In [7]:
# 实验
s1 = Single("One")
print(s1.name)

执行: __new__
执行__init__, name = One
One


In [8]:
s2 = Single("Two")
print(s2.name)

执行: __new__
执行__init__, name = Two
Two


In [9]:
print(s1.name, s2.name)

Two Two


In [10]:
s1 is s2

True

> **到这里我们就实现了一个单例，不过注意`__init__`里面是会有其它操作，每次实例化会执行__init__。**

### 4. 更改`__new__`返回的实例
> 如果`__new__`创建的是当前类的实例，会自动调用`__init__`函数。  
通过`return`语句里的`__new__`函数的第一个参数是`cls`来保证是否为当前类实例。  
如果是其他类的类名，那么实际创建返回的就是其他类的实例，其实就不会调用当前类的`__init__`函数，**也不会**调用其他类的`__init__`函数。

In [11]:
class Demo2:
    
    def __new__(cls, *args):
        print("Demo02：__new__", args)
        return Demo1.__new__(Demo1, *args)
    
    def __init__(self, *args):
        print("Demo02：__init__:", args)

In [12]:
# 发现未执行：Demo2和Demo1的__init__方法
d = Demo2(100)

Demo02：__new__ (100,)
Demo1执行: __new__ <class '__main__.Demo1'> 100
Demo1:__new__的返回值id： 4355161104


In [13]:
d

In [14]:
# 不是Demo2的实例哦！
isinstance(d, Demo2)

False

In [15]:
isinstance(d, Demo1)

True

> **通过实验发现：**  
1. 修改了`Demo2`的`__new__`，让其返回了`Demo1`的实例
2. 如果`__new__`返回了其它类的实例，那么不会执行当前类的`__init__`函数，也不会执行其它类的`__init__`函数。

### 5. 多级继承
> 在类多级继承的时候，函数调用顺序，是遵循**MRO**(`Method Resolution Order`)的，子类未实现这方法，就会去上级找，通过`__mro__`可以查看。  
1. 自身和父类中实现的`__new__`都会执行
2. `__init__`只会执行一次，遵循MRO。

#### 5-1: 三者全部实现2个方法

In [16]:
class A:
    
    def __new__(cls, *args, **kwargs):
        print("A: __new__:",)
        return super().__new__(cls)
    
    def __init__(self):
        print("A: __init__")
        

class B(A):
    
    def __new__(cls, *args, **kwargs):
        print("B: __new__:",)
        return super().__new__(cls)
    
    def __init__(self):
        print("B: __init__")
        
        
class C(B):
    
    def __new__(cls, *args, **kwargs):
        print("C: __new__:",)
        return super().__new__(cls)
    
    def __init__(self):
        print("C: __init__")

In [17]:
# 实例化
print(C.__mro__)
c1 = C()

(<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
C: __new__:
B: __new__:
A: __new__:
C: __init__


> **发现：**`__new__`方法都执行了，`__init__`方法只执行了一级。

#### 5-2: 实现部分

In [18]:
class A:
    
    def __new__(cls, *args, **kwargs):
        print("A: __new__:",)
        return super().__new__(cls)
    
    def __init__(self):
        print("A: __init__")
        

class B(A):
    
    def __new__(cls, *args, **kwargs):
        print("B: __new__:",)
        return super().__new__(cls)
    
    def __init__(self):
        print("B: __init__")
        
        
class C(B):
    
    def __init__(self):
        print("C: __init__")

In [19]:
# 实例化
print(C.__mro__)
c = C()

(<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
B: __new__:
A: __new__:
C: __init__


#### 5-3: 都不实现

In [20]:
class A:
    
    def __new__(cls, *args, **kwargs):
        print("A: __new__:",)
        return super().__new__(cls)
    
    def __init__(self):
        print("A: __init__")
        

class B(A):
    
    def __new__(cls, *args, **kwargs):
        print("B: __new__:",)
        return super().__new__(cls)
    
    def __init__(self):
        print("B: __init__")
        
        
class C(B):
    pass

In [21]:
# 实例化
print(C.__mro__)
c = C()

(<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
B: __new__:
A: __new__:
B: __init__
